# Final Project - UDF_f105w.fits

#### Here we are importing and setting up some modules

In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10.,10.]

#### Below we have a rescale image function, this will be used to rescale the data later for the 3 color false image

In [ ]:
def rescale_image(data):
    pdata_tmp = data.copy()
    m = np.nanmean(pdata_tmp)
    vplmin = m / 2.
    vpmin = np.log10(vplmin)
    vpmax = np.log10(m * 100.)
    pdata_tmp[pdata_tmp < vplmin] = vplmin
    pdata_tmp = np.log10(pdata_tmp)
    return pdata_tmp, vpmin, vpmax

#### We are now opening and reading in the data from the fits file into a data variable, here we are also using the byteswap operation

In [ ]:
hdul = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data = hdul[0].data
data = data.byteswap(inplace = True).newbyteorder()

#### The following will display the image we are dealing with

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation = 'nearest', cmap = 'gray', vmin = m - s, vmax = m + s, origin = 'lower')
plt.colorbar();
plt.savefig("final_project_tutorial_UDF_figure_1.png")

#### We are using the sep background operation to measure the background of the image

In [ ]:
bkg = sep.Background(data)

#### The following values are the mean and noise of the background object that was returned previously

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

#### We will now interpret the background as an array like before with the image

In [ ]:
bkg_image = bkg.back()

#### The following will display the background

In [ ]:
plt.imshow(bkg_image, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar()
plt.savefig("final_project_tutorial_UDF_figure_2.png")

#### Now we will interpret the background noise as an array

In [ ]:
bkg_rms = bkg.rms()

#### The following will display the background noise

In [ ]:
plt.imshow(bkg_rms, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar()
plt.savefig("final_project_tutorial_UDF_figure_3.png")

#### We are now subtracting the background from the main image

In [ ]:
data_sub = data - bkg

#### The following function will detect objects in the image and store these objects in the objects variable

In [ ]:
objects = sep.extract(data_sub, 1.5, err = bkg.globalrms)

#### We then print out how many objects were detected

In [ ]:
len(objects)

#### The following will display the main image with the detected objects circled in red

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s, np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation = 'nearest', cmap = 'gray', vmin = m - s, vmax = m + s, origin = 'lower')

for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]),
                width = 6 * objects['a'][i],
                height = 6 * objects['b'][i],
                angle = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig("final_project_tutorial_UDF_figure_4.png")

#### The following displays the many fields that the objects data type has

In [ ]:
objects.dtype.names

#### We will now use some of these fields to perform some aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err = bkg.globalrms, gain = 1.0)

#### The following will display the first ten objects that fall under the guidelines we set above

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

#### Here we are creating a histogram for the fluxes

In [ ]:
plt.hist(flux, histtype = "barstacked", edgecolor = "black", bins = 100, range = (0, .2))
plt.xlabel("fluxes")
plt.ylabel("sources")

#### I have printed the mean, median, standard deviation, and standard deviations from the mean below; I have also attempted to pinpoint the location of the largest outlier

In [ ]:
print(f"mean is                 {np.mean(flux):.20f}")
print(f"median is               {np.median(flux):.20f}")
print(f"standard deviation is   {np.std(flux):.20f}")
print(f"\nThe largest outlier is {((np.max(flux) - np.mean(flux)) / np.std(flux)):.20f} standard deviations away from the mean")
print("As for where this outlier is, I would say it is around (2050, 1350) as the noise is the highest there")

#### We are now opening and reading in data from 2 more fits for a 3 color false image

In [ ]:
hdul_125 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
data_125 = hdul_125[0].data
data_125 = data_125.byteswap(inplace = True).newbyteorder()

hdul_160 = fits.open("hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")
data_160 = hdul_160[0].data
data_160 = data_160.byteswap(inplace = True).newbyteorder()

#### Here, we are making use of the rescale image function declared earlier to get the rescaled images, minimums, and maximums; we are also limiting each of the images using the minimums and maximums

In [ ]:
d105_res, d105_min, d105_max = rescale_image(data)
d125_res, d125_min, d125_max = rescale_image(data_125)
d160_res, d160_min, d160_max = rescale_image(data_160)

d105_res[d105_res < d105_min] = d105_min
d105_res[d105_res > d105_max] = d105_max
d125_res[d125_res < d125_min] = d125_min
d125_res[d125_res > d125_max] = d125_max
d160_res[d160_res < d160_min] = d160_min
d160_res[d160_res > d160_max] = d160_max

#### The following section creates the final image using the 3 previous rescaled images

In [ ]:
rgb = np.zeros((d105_res.shape[0], d105_res.shape[1], 3))
rgb[:,:,0] = (d160_res - d160_min) / (d160_max - d160_min)
rgb[:,:,1] = (d125_res - d125_min) / (d125_max - d125_min)
rgb[:,:,2] = (d105_res - d105_min) / (d105_max - d105_min)

#### Finally, the final image is being displayed and saved as a png

In [ ]:
f, ax = plt.subplots(1, 1, figsize = (10., 10.))
ax.axis('off')
ax.imshow(rgb)
plt.savefig("final_project_3_color_false_image.png", bbox_inches = "tight", pad_inches = 0, dpi = 600)